In [1]:
%load_ext autoreload
%autoreload 2

In [40]:
from Corpora import MovieReviewCorpus
from Lexicon import SentimentLexicon
from Statistics import SignTest
from Classifiers import NaiveBayesText, SVMText
from Extensions import SVMDoc2Vec

import numpy as np

In [3]:
# retrieve corpus
corpus=MovieReviewCorpus(stemming=False,pos=False)

# use sign test for all significance testing
signTest=SignTest()

print("--- classifying reviews using sentiment lexicon  ---")

# read in lexicon
lexicon=SentimentLexicon()

# on average there are more positive than negative words per review (~7.13 more positive than negative per review)
# to take this bias into account will use threshold (roughly the bias itself) to make it harder to classify as positive
threshold=8

# question 0.1
lexicon.classify(corpus.reviews,threshold,magnitude=False)
token_preds=lexicon.predictions
print(f"token-only results: {lexicon.getAccuracy():.2f}")

lexicon.classify(corpus.reviews,threshold,magnitude=True)
magnitude_preds=lexicon.predictions
print(f"magnitude results: {lexicon.getAccuracy():.2f}")

# question 0.2
p_value=signTest.getSignificance(token_preds,magnitude_preds)
significance = "significant" if p_value < 0.05 else "not significant"
print(f"magnitude lexicon results are {significance} with respect to token-only")

--- classifying reviews using sentiment lexicon  ---
token-only results: 0.67
magnitude results: 0.68
magnitude lexicon results are not significant with respect to token-only


In [4]:
# question 1.0
print("--- classifying reviews using Naive Bayes on held-out test set ---")
NB=NaiveBayesText(smoothing=False,bigrams=False,trigrams=False,discard_closed_class=False)
NB.train(corpus.train)
NB.test(corpus.test)
# store predictions from classifier
non_smoothed_preds=NB.predictions
print(f"Accuracy without smoothing: {NB.getAccuracy():.2f}")

--- classifying reviews using Naive Bayes on held-out test set ---
Accuracy without smoothing: 0.46


In [5]:
# question 2.0
# use smoothing
NB=NaiveBayesText(smoothing=True,bigrams=False,trigrams=False,discard_closed_class=False)
NB.train(corpus.train)
NB.test(corpus.test)
smoothed_preds=NB.predictions
# saving this for use later
num_non_stemmed_features=len(NB.vocabulary)
print(f"Accuracy using smoothing: {NB.getAccuracy():.2f}")


# question 2.1
# see if smoothing significantly improves results
p_value=signTest.getSignificance(non_smoothed_preds,smoothed_preds)
significance = "significant" if p_value < 0.05 else "not significant"
print(f"results using smoothing are {significance} with respect to no smoothing")

Accuracy using smoothing: 0.82
results using smoothing are significant with respect to no smoothing


In [6]:
# question 3.0
print("--- classifying reviews using 10-fold cross-evaluation ---")
# using previous instantiated object
NB.crossValidate(corpus)
# using cross-eval for smoothed predictions from now on
smoothed_preds=NB.predictions
print(f"Accuracy: {NB.getAccuracy():.3f}")
print(f"Std. Dev: {NB.getStdDeviation()}")

--- classifying reviews using 10-fold cross-evaluation ---
Accuracy: 0.810
Std. Dev: 0.020736441353327688


In [7]:
# question 4.0
print("--- stemming corpus ---")
# retrieve corpus with tokenized text and stemming (using porter)
stemmed_corpus=MovieReviewCorpus(stemming=True,pos=False)
print("--- cross-validating NB using stemming ---")
NB.crossValidate(stemmed_corpus)
stemmed_preds=NB.predictions
print(f"Accuracy: {NB.getAccuracy():.3f}")
print(f"Std. Dev: {NB.getStdDeviation():.3f}")

# TODO Q4.1
# see if stemming significantly improves results on smoothed NB
p_value=signTest.getSignificance(stemmed_preds,smoothed_preds)
significance = "significant" if p_value < 0.05 else "not significant"
print(f"results using smoothing are {significance} with respect to no smoothing")

# TODO Q4.2
print("--- determining the number of features before/after stemming ---")
print("Number of features before stemming:", num_non_stemmed_features) #variable set previously
NB.train(stemmed_corpus.train)
print("Number of features after stemming:", len(NB.vocabulary))

--- stemming corpus ---
--- cross-validating NB using stemming ---
Accuracy: 0.814
Std. Dev: 0.026
results using smoothing are not significant with respect to no smoothing
--- determining the number of features before/after stemming ---
Number of features before stemming: 52555
Number of features after stemming: 32404


In [8]:
# question Q5.0
# cross-validate model using smoothing and bigrams
print("--- cross-validating naive bayes using smoothing and bigrams ---")
NB=NaiveBayesText(smoothing=True,bigrams=True,trigrams=False,discard_closed_class=False)
NB.crossValidate(corpus)
smoothed_and_bigram_preds=NB.predictions
print(f"Accuracy: {NB.getAccuracy():.2f}") 
print(f"Std. Dev: {NB.getStdDeviation():.2f}")
# counting features using bigrams
NB.train(corpus.train)
num_bigram_features = len(NB.vocabulary)

# see if bigrams significantly improves results on smoothed NB only
p_value=signTest.getSignificance(smoothed_preds,smoothed_and_bigram_preds)
signifance = "significant" if p_value < 0.05 else "not significant"
print(f"results using smoothing and bigrams are {signifance} with respect to smoothing only")


print("--- cross-validating naive bayes using smoothing and trigrams ---")
NB=NaiveBayesText(smoothing=True,bigrams=False,trigrams=True,discard_closed_class=False)
NB.crossValidate(corpus)
smoothed_and_trigram_preds=NB.predictions
print(f"Accuracy: {NB.getAccuracy():.2f}") 
print(f"Std. Dev: {NB.getStdDeviation():.2f}")
# counting features using trigrams
NB.train(corpus.train)
num_trigram_features = len(NB.vocabulary)

# see if bigrams significantly improves results on smoothed NB only
p_value=signTest.getSignificance(smoothed_preds,smoothed_and_trigram_preds)
signifance = "significant" if p_value < 0.05 else "not significant"
print(f"results using smoothing and trigrams are {signifance} with respect to smoothing only")



# TODO Q5.1
print("--- determining the number of features before/after using bigrams ---")
print("Number of features before using bigrams (Q3):", num_non_stemmed_features) #variable set previously
print("Number of features after using bigrams:", num_bigram_features)
print("Number of features after using trigrams:", num_trigram_features)



--- cross-validating naive bayes using smoothing and bigrams ---
Accuracy: 0.76
Std. Dev: 0.02
results using smoothing and bigrams are significant with respect to smoothing only
--- cross-validating naive bayes using smoothing and trigrams ---
Accuracy: 0.70
Std. Dev: 0.03
results using smoothing and trigrams are significant with respect to smoothing only
--- determining the number of features before/after using bigrams ---
Number of features before using bigrams (Q3): 52555
Number of features after using bigrams: 500086
Number of features after using trigrams: 1015074


In [28]:
# TODO Q6 and 6.1

# retrieve corpus
corpus=MovieReviewCorpus(stemming=False,pos=False)

print("--- classifying reviews using SVM 10-fold cross-eval ---")
SVM=SVMText(bigrams=False,trigrams=False,discard_closed_class=False)
SVM.crossValidate(corpus)
print(f"Accuracy: {SVM.getAccuracy():.4f}") 
print(f"Std. Dev: {SVM.getStdDeviation():.3f}")

--- classifying reviews using SVM 10-fold cross-eval ---
Accuracy: 0.8380
Std. Dev: 0.020


In [29]:
# TODO Q7

print("--- adding in POS information to corpus ---")
# retrieve corpus with POS information
corpus_pos = MovieReviewCorpus(stemming=False,pos=True)

print("--- training svm on word+pos features ----")
SVM=SVMText(bigrams=False,trigrams=False,discard_closed_class=False)
SVM.crossValidate(corpus_pos)
print(f"Accuracy: {SVM.getAccuracy():.4f}") 
print(f"Std. Dev: {SVM.getStdDeviation():.3f}")

print("--- training svm discarding closed-class words ---")
SVM_dcc=SVMText(bigrams=False,trigrams=False,discard_closed_class=True)
SVM_dcc.crossValidate(corpus_pos)
print(f"Accuracy: {SVM_dcc.getAccuracy():.4f}") 
print(f"Std. Dev: {SVM_dcc.getStdDeviation():.3f}")

--- adding in POS information to corpus ---
--- training svm on word+pos features ----
Accuracy: 0.8345
Std. Dev: 0.026
--- training svm discarding closed-class words ---
Accuracy: 0.8360
Std. Dev: 0.023


## Question 8 (40% of the marks)

In [42]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import tensorflow as tf

In [37]:
print("--- reading dataset from keras.datasets ---")
imdb = tf.keras.datasets.imdb
(train_reviews, train_labels), (test_reviews, test_labels) = imdb.load_data()

print("--- getting vocabulary ---")
vocab_word_to_id = imdb.get_word_index()
vocab_word_to_id = {word:(word_id + 3) for word, word_id in vocab_word_to_id.items()}
vocab_word_to_id["<PAD>"] = 0
vocab_word_to_id["<START>"] = 1
vocab_word_to_id["<UNK>"] = 2
vocab_word_to_id["<UNUSED>"] = 3

vocab_id_to_word =  dict([(value, key) for (key, value) in vocab_word_to_id.items()])

--- reading dataset from keras.datasets ---


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.9/site-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.9/site-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do t

--- getting vocabulary ---
1646592/1641221 [==============================] - 0s 0us/step


In [38]:
def decode_int_review(int_review):
    return [vocab_id_to_word.get(id, "?") for id in int_review]

In [43]:
print("--- concatenating train and test reviews to infer their embeddings at once ---")
reviews = np.concatenate((train_reviews, test_reviews))

print("--- creating documents after decoding reviews ---")
docs = [TaggedDocument(decode_int_review(int_review), [i]) for i, int_review in enumerate(reviews)]

--- concatenating train and test reviews to infer their embeddings at once ---
--- creating documents after decoding reviews ---


In [45]:
# question 8.0
print("--- pre-training doc2vec model ---")
d2v_model = Doc2Vec(docs, dm=0, min_count=2, vector_size=100, hs=0, negative=5, epochs=100, sample=0)

--- pre-training doc2vec model ---


In [47]:
d2v_model.save("d2v_model.model")

In [75]:
print("--- extracting learned embeddings and splitting them into training and testing sets")
embeddings = d2v_model.dv.get_normed_vectors()
train_embeddings, test_embeddings = np.split(embeddings, [25000]) # 25K because we have 25K reviews for training and 25K for testing

print("training shape:", train_embeddings.shape)
print("testing shape:", test_embeddings.shape)

--- extracting learned embeddings and splitting them into training and testing sets
training shape: (25000, 100)
testing shape: (25000, 100)


In [78]:
print("--- using document embeddings ---")

SVMD2V = SVMDoc2Vec(None)
SVMD2V.train(train_embeddings, train_labels)
SVMD2V.test(test_embeddings, test_labels)

print(f"Accuracy: {SVMD2V.getAccuracy():.2f}") 
print(f"Std. Dev: {SVMD2V.getStdDeviation():.2f}")

--- using document embeddings ---
Accuracy: 0.90
Std. Dev: 0.01
